# 如何使用时间加权向量存储检索器
该[检索器](/docs/concepts/retrievers/)结合了语义[相似度](/docs/concepts/embedding_models/#measure-similarity)与时间衰减机制。
评分算法如下：
好的,我将按照要求进行翻译,只输出翻译后的中文markdown内容,不显示任何额外信息。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个示例markdown文档的翻译。

## 主要功能

- 准确翻译英文内容
- 保持原始markdown格式
- 支持多种文档类型

### 使用说明

1. 提供需要翻译的英文markdown内容
2. 系统会自动处理并返回中文版本
3. 格式会与原文完全一致

> 注意:翻译会严格保持原始文档结构

**重要提示**:请不要包含任何额外的解释或说明文字,只需输出翻译后的markdown内容本身。

[这是一个链接示例](https://example.com)

```python
# 代码块会原样保留
def example():
    print("Hello World")
```语义相似度 + (1.0 - 衰减率) ^ 已过小时数好的,我将按照要求进行翻译,只输出翻译后的中文markdown内容:

# 欢迎使用翻译助手

这是一个Markdown格式的翻译示例:

## 主要功能
- 提供高质量的文本翻译
- 保持原始格式不变
- 支持多种文档类型

### 使用说明
1. 输入需要翻译的文本
2. 指定目标语言
3. 获取翻译结果

> 注意:翻译质量取决于原文的清晰度和复杂度

```python
# 示例代码
def translate(text):
    return "翻译结果"
```

**重要提示**: 请确保输入文本没有敏感信息

| 特性 | 描述 |
|------|------|
| 准确性 | 高精度翻译 |
| 速度 | 实时响应 |
| 支持格式 | Markdown/HTML/Plain text |
值得注意的是，`hours_passed` 指的是自检索器中对象**最后一次被访问以来**经过的小时数，而非自其创建以来的时间。这意味着频繁被访问的对象会保持“新鲜”状态。

In [1]:
from datetime import datetime, timedelta

import faiss
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

## 低衰减率
较低的`衰减率`（在极端情况下，我们会将其设置为接近0）意味着记忆会被"记住"更长时间。当`衰减率`为0时，记忆永远不会被遗忘，这使得该检索器等同于向量查找。

In [2]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

In [3]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents(
    [Document(page_content="hello world", metadata={"last_accessed_at": yesterday})]
)
retriever.add_documents([Document(page_content="hello foo")])

['73679bc9-d425-49c2-9d74-de6356c73489']

In [4]:
# "Hello World" is returned first because it is most salient, and the decay rate is close to 0., meaning it's still recent enough
retriever.invoke("hello world")

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 10, 22, 16, 37, 40, 818583), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 37, 975074), 'buffer_idx': 0}, page_content='hello world')]

## 高衰减率
当`衰减率`较高时（例如多个9），`时效性分数`会迅速降至0！如果将该值直接设为1，所有对象的`时效性`都将为0，这再次使其等同于向量查找。


In [5]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

In [6]:
yesterday = datetime.now() - timedelta(days=1)
retriever.add_documents(
    [Document(page_content="hello world", metadata={"last_accessed_at": yesterday})]
)
retriever.add_documents([Document(page_content="hello foo")])

['379631f0-42c2-4773-8cc2-d36201e1e610']

In [7]:
# "Hello Foo" is returned first because "hello world" is mostly forgotten
retriever.invoke("hello world")

[Document(metadata={'last_accessed_at': datetime.datetime(2024, 10, 22, 16, 37, 46, 553633), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 43, 927429), 'buffer_idx': 1}, page_content='hello foo')]

## 虚拟时间
借助LangChain中的一些工具，你可以模拟时间组件。

In [8]:
from langchain_core.utils import mock_now

In [9]:
# Notice the last access time is that date time

tomorrow = datetime.now() + timedelta(days=1)

with mock_now(tomorrow):
    print(retriever.invoke("hello world"))

[Document(metadata={'last_accessed_at': MockDateTime(2024, 10, 23, 16, 38, 19, 66711), 'created_at': datetime.datetime(2024, 10, 22, 16, 37, 43, 599877), 'buffer_idx': 0}, page_content='hello world')]
